### 6.1 语言模型
* 语言模型的计算
* n元语法
    - 通过马尔可夫假设简化了语言模型的计算
    - 假设：一个词的出现只与前面n个词相关，即n阶马尔可夫链
    - unigram，bigram，trigram

### 6.2 循环神经网络
* 不含隐藏状态的神经网络
* 含隐藏状态的循环神经网络
    - 保存上一时间步的隐藏变量H（t-1），
    - 并引入新的权重参数Whh，描述如何使用上一时间步的隐藏变量
* 循环神经网络的隐藏状态可以捕捉至当前时间步的序列的历史信息
* 循环神经网络模型参数的数量不随时间步的增加而增长
* 可以基于字符级循环神经网络来创建语言模型

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
import time
import d2lzh_pytorch as d2l
import math

In [ ]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

In [ ]:
torch.matmul(torch.cat((X, H), dim=1), 
             torch.cat((W_xh, W_hh), dim=0))

### 6.3 语言模型数据集（周杰伦专辑歌词）

In [ ]:
# 读取数据集
import random
import zipfile

data_path = 'data/jaychou_lyrics.txt.zip'
with zipfile.ZipFile(data_path) as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
print(len(corpus_chars))
corpus_chars[:40]

In [ ]:
corpus_chars = corpus_chars.replace('\n', ' '
        ).replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [ ]:
# 建立字符索引
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in 
                   enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

In [ ]:
corpus_indices = [char_to_idx[char] 
                  for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:',''.join([idx_to_char[idx] 
                        for idx in sample]))
print('indices:', sample)

##### 时序数据的采样
* 随机采样
    - 每次从数据里随机采样一个小批量
    - 每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻
    - 因此，无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态
    - 训练模型时，每次随机采样前都需要重新初始化隐藏状态
* 相邻采样
    - 令相邻的两个随机小批量在原始序列上的位置相毗邻

In [ ]:
# 随机采样
def data_iter_random(corpus_indices, batch_size, 
                     num_steps, device=None):
    ''' 每次从数据里随机采样一个小批量
        batch_size：每个小批量的样本数
        num_steps：每个样本所包含的时间步数
    '''
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    
    if device is None:
        device = torch.device('cuda' if 
                torch.cuda.is_available() else 'cpu')
        
    for i in range(epoch_size):
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in 
                 batch_indices]
        yield torch.tensor(X, dtype=torch.float32, 
                           device=device),  torch.tensor(
                Y, dtype=torch.float32, device=device)

In [ ]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, 
                             num_steps=6):
    print("X:", X, '\nY:', Y, '\n')

In [ ]:
# 相邻采样
def data_iter_consecutive(corpus_indices, batch_size,
                         num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if 
                torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, 
                                 dtype=torch.float32,
                                 device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    #print(data_len, batch_len)
    indices = corpus_indices[0: batch_size * batch_len
                            ].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [ ]:
my_seq = list(range(30))
for X, Y in data_iter_consecutive(my_seq, batch_size=2,
                                 num_steps=6):
    print('X:', X, '\nY:', Y, '\n')

### 6.4 循环神经网络的从零开始实现
从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作

In [ ]:
# one-hot向量
def one_hot(x, n_class, dtype=torch.float32):
    # x shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype,
                     device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

x = torch.tensor([0, 2])
one_hot(x, vocab_size)

In [ ]:
def to_onehot(X, n_class):
    # X shape: (batch, seq_len)
    # output: seq_len elsements of (batch, n_class)
    return [one_hot(X[:, i], n_class) 
            for i in range(X.shape[1])]

In [ ]:
X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, 10)
print(len(inputs), inputs[0].shape)

In [ ]:
print(X)

In [ ]:
print(inputs)

In [ ]:
# 初始化模型参数
def get_params():
    def _one(shape):
        ts = torch.tensor(
            np.random.normal(0, 0.01, size=shape), 
            device=device, 
            dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, 
            device=device, requires_grad=True))
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,
            device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

In [ ]:
# 定义模型
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), 
                        device=device),)

In [ ]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + 
                      torch.matmul(H, W_hh) +
                      b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [ ]:
num_inputs, num_hiddens = vocab_size, 256
num_outputs = vocab_size
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

In [ ]:
def predict_rnn(prefix, num_chars, rnn, params, 
                init_rnn_state, num_hiddens, vocab_size,
               device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], 
                device=device), vocab_size)
        # 计算输出，更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [ ]:
# 裁剪梯度
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

In [ ]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state,
                         num_hiddnes, vocab_size, 
                         device, corpus_indices, 
                         idx_to_char, char_to_idx,
                         is_random_iter, num_epochs,
                         num_steps, lr, clipping_theta, 
                         batch_size, pred_period, pred_len,
                         prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, 
                                   num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, 
                                 batch_size, num_steps,
                                 device)
        for X, Y in data_iter:
            if is_random_iter:
                # 如果随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, 
                                       num_hiddens, 
                                       device)
            else:
                # 否则需要使用detach函数从计算图分离隐藏状态
                # 这样为了使模型参数的梯度计算只依赖一次迭代
                # 防止梯度计算开销太大
                for s in state:
                    s.detach_()
            
            inputs = to_onehot(X, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            outputs = torch.cat(outputs, dim=0)
            y = torch.transpose(Y, 0, 1).contiguous(
                    ).view(-1)
            l = loss(outputs, y.long())
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
                    
            l.backward()
            grad_clipping(params, clipping_theta, device)
            d2l.sgd(params, lr, l)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' 
                  % (epoch + 1, math.exp(l_sum / n), 
                 time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len,
                    rnn, params, init_rnn_state, 
                    num_hiddens, vocab_size, device,
                    idx_to_char, char_to_idx))

In [ ]:
num_epochs, num_steps, batch_size = 250, 35, 32
lr, clipping_theta = 1e2, 1e-2
pred_period, pred_len = 50, 50
prefixes = ['分开', '不分开']

In [ ]:
train_and_predict_rnn(rnn, get_params, init_rnn_state,
                     num_hiddens, vocab_size, device,
                     corpus_indices, idx_to_char,
                     char_to_idx, True, num_epochs,
                     num_steps, lr, clipping_theta,
                     batch_size, pred_period, pred_len,
                     prefixes)

### 6.5 循环神经网络的简洁实现

In [ ]:
num_hiddens = 256
rnn_layer = nn.RNN(input_size=vocab_size, 
                   hidden_size=num_hiddens)

In [ ]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if
                rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, 
                               vocab_size)
        self.state = None
    
    def forward(self, inputs, state): 
        # inputs: (batch, seq_len)
        X = to_onehot(inputs, self.vocab_size)
        # X: (seq_len, batch, vocab_size)
        Y, self.state = self.rnn(torch.stack(X), state)
        # Y: (seq_len, batch, hidden)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [ ]:
def predict_rnn_pytorch(prefix, num_chars, model, 
                        vocab_size, device, idx_to_char,
                       char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device
                        ).view(1, 1)
        if state is not None:
            if isinstance(state, tuple):
                state = (state[0].to(device),
                        state[1].to(device))
            else:
                state = state.to(device)
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
            
    return ''.join([idx_to_char[i] for i in output])

In [ ]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device,
                   idx_to_char, char_to_idx)

In [ ]:
def train_and_predict_rnn_pytorch(model, num_hiddens,
        vocab_size, device, corpus_indices, idx_to_char,
        char_to_idx, num_epochs, num_steps, lr, 
        clipping_theta, batch_size, pred_period, pred_len,
        prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, 
                batch_size, num_steps, device)
        for X, Y in data_iter:
            if state is not None:
                if isinstance(state, tuple):
                    state = (state[0].detach(), 
                            state[1].detach())
                else:
                    state = state.detach()
            (output, state) = model(X, state)
            # output shape: 
            # (num_steps * batch_size, vocab_size)
            y = torch.transpose(Y, 0, 1).contiguous(
                    ).view(-1)
            # shape: batch * num_steps 与y一致
            l = loss(output, y.long())
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), 
                         clipping_theta,
                         device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec'
                 % (epoch + 1, perplexity, 
                  time.time() - start))
            for prefix in prefixes:
                print('-', predict_rnn_pytorch(prefix,
                        pred_len, model, vocab_size,
                        device, idx_to_char, char_to_idx)
                     )

In [ ]:
num_epochs, batch_size = 250, 32 
lr, clipping_theta = 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, 
        vocab_size, device, corpus_indices, idx_to_char,
        char_to_idx, num_epochs, num_steps, lr, 
        clipping_theta, batch_size, pred_period, pred_len,
        prefixes)

### 6.6 通过时间反向传播
* back-propagation through time
* 需要将循环神经网络按时间步展开，从而得到模型变量和参数之间的依赖关系，并依据链式法则应用反向传播计算并存储梯度
* 当总的时间步数较大或者当前时间步较小时，循环神经网络的梯度容易出现衰减或爆炸
* 可以通过裁剪梯度应对梯度爆炸

### 6.7 门控循环单元（GRU）
* 为了更好的捕捉时间序列中时间步距离较大的依赖关系
* 重置门和更新门 （reset gate，update gate）
    - 重置门有助于捕捉时间序列里短期的依赖关系
    - 更新门有助于捕捉时间序列里长期的依赖关系

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size
    ) = d2l.load_data_jay_lyrics()

In [ ]:
num_inputs, num_hiddens = vocab_size, 256
num_outputs = vocab_size

In [ ]:
def get_params():
    def _one(shape):
        ts = torch.tensor(
            np.random.normal(0, 0.01, size=shape), 
            device=device,
            dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               torch.nn.Parameter(
                   torch.zeros(num_hiddens, 
                               device=device, 
                               dtype=torch.float32), 
                   requires_grad=True)
               )
    W_xz, W_hz, b_z = _three() # update gate
    W_xr, W_hr, b_r = _three() # reset gate
    W_xh, W_hh, b_h = _three() # hidden state
    
    # output
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(
                torch.zeros(num_outputs, 
                            device=device, 
                            dtype=torch.float32), 
                requires_grad=True)
    
    return nn.ParameterList([
        W_xz, W_hz, b_z,
        W_xr, W_hr, b_r,
        W_xh, W_hh, b_h,
        W_hq, b_q
    ]) 

In [ ]:
def init_gru_state(batch_size, num_hidden, device):
    return (torch.zeros((batch_size, num_hiddens), 
                        device=device), )

In [ ]:
def gru(inputs, state, params):
    (W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q) = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + 
                         torch.matmul(H, W_hz) +
                         b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) +
                         torch.matmul(H, W_hr) +
                         b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + 
                            R * torch.matmul(H, W_hh) +
                            b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [ ]:
num_epochs, num_steps, batch_size = 160, 35, 32
lr, clipping_theta = 1e2, 1e-2
pred_period, pred_len = 40, 50
prefixes = ['分开', '不分开']

In [ ]:
train_and_predict_rnn(gru, get_params, init_gru_state,
                     num_hiddens, vocab_size, device,
                     corpus_indices, idx_to_char,
                     char_to_idx, False, num_epochs,
                     num_steps, lr, clipping_theta,
                     batch_size, pred_period, pred_len,
                     prefixes)

In [ ]:
# 简洁实现
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, 
                   hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
'''
train_and_predict_rnn_pytorch(model, num_hiddens, 
        vocab_size, device, corpus_indices, idx_to_char,
        char_to_idx, num_epochs, num_steps, lr, 
        clipping_theta, batch_size, pred_period, pred_len,
        prefixes)
'''

### 6.8 长短期记忆（LSTM）
* 输入门，遗忘门和输出门
* 候选记忆细胞
* 隐藏状态